**Define imports and constants**

In [1]:
import numpy as np
import pandas as pd
import os
import gc
import cv2
import tensorflow as tf
import seaborn as sn
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from tensorflow.keras.callbacks import *
from keras.models import Model
from keras.utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import backend as K
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

print(tf.__version__)

2.2.0


Using TensorFlow backend.


In [2]:
training = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
print(training.head(10))

     image_name  patient_id     sex  age_approx anatom_site_general_challenge  \
0  ISIC_2637011  IP_7279968    male        45.0                     head/neck   
1  ISIC_0015719  IP_3075186  female        45.0               upper extremity   
2  ISIC_0052212  IP_2842074  female        50.0               lower extremity   
3  ISIC_0068279  IP_6890425  female        45.0                     head/neck   
4  ISIC_0074268  IP_8723313  female        55.0               upper extremity   
5  ISIC_0074311  IP_2950485  female        40.0               lower extremity   
6  ISIC_0074542  IP_4698288    male        25.0               lower extremity   
7  ISIC_0075663  IP_6017204  female        35.0                         torso   
8  ISIC_0075914  IP_7622888    male        30.0                         torso   
9  ISIC_0076262  IP_5075533  female        50.0               lower extremity   

  diagnosis benign_malignant  target  
0   unknown           benign       0  
1   unknown           benign  

In [3]:
print(training["target"].value_counts())

0    32542
1      584
Name: target, dtype: int64


In [4]:
train_aug = ImageDataGenerator(rescale=1./255,
                     rotation_range=70,
                     width_shift_range=0.3, 
                     height_shift_range=0.3,
                     shear_range=0.1,
                     channel_shift_range=20,
                     horizontal_flip=True,
                     vertical_flip=True)

val_aug = ImageDataGenerator(rescale=1./255)

In [5]:
def focal_loss(alpha=0.25,gamma=2.0):
    def focal_crossentropy(y_true, y_pred):
        bce = K.binary_crossentropy(y_true, y_pred)
        
        y_pred = K.clip(y_pred, K.epsilon(), 1.- K.epsilon())
        p_t = (y_true*y_pred) + ((1-y_true)*(1-y_pred))
        
        alpha_factor = 1
        modulating_factor = 1

        alpha_factor = y_true*alpha + ((1-alpha)*(1-y_true))
        modulating_factor = K.pow((1-p_t), gamma)

        # compute the final loss and return
        return K.mean(alpha_factor*modulating_factor*bce, axis=-1)
    return focal_crossentropy

In [6]:
!pip install git+https://github.com/qubvel/efficientnet

  Cloning https://github.com/qubvel/efficientnet to /tmp/pip-req-build-vmi_okq7
  Running command git clone -q https://github.com/qubvel/efficientnet /tmp/pip-req-build-vmi_okq7
     |████████████████████████████████| 50 kB 1.9 MB/s eta 0:00:011
  Created wheel for efficientnet: filename=efficientnet-1.1.0-py3-none-any.whl size=18397 sha256=3b910ef724ecc84b12f208a9345d9bf0738011ea9034bcda176b2bf3e0091421
  Stored in directory: /tmp/pip-ephem-wheel-cache-b80xkc1v/wheels/11/69/85/814d64d694c96db0eef17b718042d644a1e54f113920481920
Successfully built efficientnet


In [7]:
import efficientnet.keras as efn 

In [8]:
opt = keras.optimizers.Adam()
# opt = keras.optimizers.Adamax(learning_rate=1e-5)
# opt = keras.optimizers.Adagrad(learning_rate=1e-5)
loss = keras.losses.BinaryCrossentropy(label_smoothing = 0.02)
# loss = focal_loss()
auc = keras.metrics.AUC()

In [9]:
def lr_callback():   
    def lrfn(epoch, rate):
        if epoch < 3:
            lr = 1e-4           
        else:
            lr = rate*0.75            
        return lr

    callback = LearningRateScheduler(lrfn, verbose=True)
    return callback

**Begin Training Cycle**

In [10]:
model_num = 0

In [11]:
m = training[training["target"]==1]
b = training[training["target"]==0].sample(2000)
df = pd.concat([m,b])
df.reset_index(inplace=True)
df.drop(labels=["index", "patient_id", "sex", "age_approx", "anatom_site_general_challenge", "diagnosis", "benign_malignant"], axis=1, inplace=True)
df["image_name"] = "../input/images-siim-512x512/train/train_512x512/" + df["image_name"].astype(str) + ".jpg"
df.head()

,image_name,target
0,../input/images-siim-512x512/train/train_512x5...,1
1,../input/images-siim-512x512/train/train_512x5...,1
2,../input/images-siim-512x512/train/train_512x5...,1
3,../input/images-siim-512x512/train/train_512x5...,1
4,../input/images-siim-512x512/train/train_512x5...,1


In [12]:
imgs = []
labels = []
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    img = cv2.imread(str(row["image_name"]))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    imgs.append(img)
    labels.append(row["target"])
imgs = np.array(imgs)
labels = np.array(labels)
print(imgs.shape)
print(labels.shape)
del m, b, df
gc.collect()


(2584, 512, 512, 3)
(2584,)


4

In [13]:
trainX, valX, trainY, valY = train_test_split(imgs, labels, test_size=0.2, random_state=888)
print(trainX.shape)
print(valX.shape)
print(trainY.shape)
print(valY.shape)
lenT = trainX.shape[0]
lenV = valX.shape[0]
del imgs, labels
gc.collect()

(2067, 512, 512, 3)
(517, 512, 512, 3)
(2067,)
(517,)


20

In [14]:
train_gen = train_aug.flow(trainX, trainY, batch_size = 4, shuffle = True)
val_gen = val_aug.flow(valX, valY, batch_size = 4, shuffle = False)
del trainX, valX, trainY, valY
gc.collect()

20

In [15]:
early_stop = EarlyStopping(monitor='val_loss', patience=2)
checkpoint = ModelCheckpoint("{val_loss:.3f}-{epoch:02d}.hdf5",monitor = 'val_loss',verbose = 1,save_best_only = True,mode = 'min')

callbacks = [early_stop, checkpoint, lr_callback()]

In [16]:
def eff():
    if(model_num == 0):
        return efn.EfficientNetB6(
            include_top=False,
            weights="imagenet",
            input_shape=(512,512,3)
        )
    else: 
        return efn.EfficientNetB7(
            include_top=False,
            weights="imagenet",
            input_shape=(512,512,3)
        )
# effnet.trainable=False
effnet=eff()
flat = Flatten()(effnet.output)
# gap = GlobalAveragePooling2D()(effnet.output)
# final = Dropout(0.4)(flat)#
# final = Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(final)
# final = Dropout(0.4)(final)
dropout = Dropout(0.2)(flat)
final = Dense(1, activation="sigmoid")(dropout)
model = Model(effnet.input,final)
model.summary()

165527552/165527152 [==============================] - 7s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 256, 256, 56) 1512        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 256, 256, 56) 224         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 256, 256, 56) 0           stem_bn[0][0]                    
___________________________

Total params: 41,549,961
Trainable params: 41,325,529
Non-trainable params: 224,432
__________________________________________________________________________________________________


In [17]:
model.compile(loss=loss, metrics=auc, optimizer=opt)

In [ ]:
history = model.fit_generator(
    train_gen,
    steps_per_epoch = lenT // 4,
    epochs = 8, 
    validation_data = val_gen,
    validation_steps = lenV // 4,
    callbacks = callbacks
)


Epoch 00001: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 1/8
516/516 [==============================] - ETA: 0s - loss: 0.8785 - auc: 0.6742
Epoch 00002: val_loss improved from 0.92860 to 0.68210, saving model to 0.682-02.hdf5
516/516 [==============================] - 346s 671ms/step - loss: 0.8785 - auc: 0.6742 - val_loss: 0.6821 - val_auc: 0.7671 - lr: 1.0000e-04

Epoch 00003: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 3/8
516/516 [==============================] - ETA: 0s - loss: 0.8916 - auc: 0.6718
Epoch 00003: val_loss did not improve from 0.68210
516/516 [==============================] - 343s 665ms/step - loss: 0.8916 - auc: 0.6718 - val_loss: 2.9798 - val_auc: 0.7976 - lr: 1.0000e-04

Epoch 00004: LearningRateScheduler reducing learning rate to 7.499999810534064e-05.
Epoch 4/8
 45/516 [=>............................] - ETA: 4:53 - loss: 0.7494 - auc: 0.7346

In [ ]:
print("DONE")

In [ ]:
# model.save('effnet2.h5')

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch

plt.figure()
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.plot(hist['epoch'], hist['loss'], label='Train Error')
plt.plot(hist['epoch'], hist['val_loss'], label='Val Error')
plt.ylim([0, 1])
plt.legend()

plt.figure()
plt.xlabel('Epoch')
plt.ylabel('accuracy')
plt.plot(hist['epoch'], hist['auc'], label='Train Acc')
plt.plot(hist['epoch'], hist['val_auc'], label='Val Acc')
plt.ylim([0, 1])
plt.legend()
plt.show()

In [ ]:
del train_gen, val_gen
gc.collect()

In [ ]:
del effnet, flat, dropout, final, model, hist, history
gc.collect()

In [ ]:
for layer in effnet.layers[745:]:
   layer.trainable = True
for i, layer in enumerate(effnet.layers):
   print(i, layer.name, layer.trainable)

In [ ]:
model.summary()

And now there's several million more trainable parameters.

I don't use AUC in this demo, but to implement it all you need to do is to set metrics=\[auc\]

In [ ]:
model.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer=opt)

history = model.fit_generator(
    train_gen,
    steps_per_epoch = trainX.shape[0] // 16,
    epochs = 10, 
    validation_data = val_gen,
    validation_steps = valX.shape[0] // 16,
    callbacks = callbacks
)

In [ ]:
del trainX, trainY, valX, valY
gc.collect()

Now for predicting. I print out the first 50 predictions just to make sure things are going right.

In [ ]:
test_images = np.load('../input/siimisic-melanoma-resized-images/x_test_224.npy')
print(test_images.shape)

In [ ]:
def predict():
    predictions = np.empty(0)
    k = test_images.shape[0]
    i = 0
    p_bar = tqdm(total=test_images.shape[0])
    while True:
        images=[]
        for j in range(64):
            p_bar.update(1)
            if(i>=k):
                pred=model.predict(np.array(images))
                pred=np.squeeze(pred)
                predictions = np.concatenate((predictions, pred))
                print(predictions.shape)
                return predictions
            images.append(test_images[i].astype(np.float32)/255.) 
            i+=1
            
        pred=model.predict(np.array(images))
        pred=np.squeeze(pred)
        predictions = np.concatenate((predictions, pred))
        del images
        gc.collect()
        
predictions = predict()

In [ ]:
def predict1():
    predictions = np.empty(0)
    k = test_images.shape[0]
    i = 0
    p_bar = tqdm(total=test_images.shape[0])
    while True:
        images=[]
        for j in range(64):
            p_bar.update(1)
            if(i>=k):
                pred=model1.predict(np.array(images))
                pred=np.squeeze(pred)
                predictions = np.concatenate((predictions, pred))
                print(predictions.shape)
                return predictions
            images.append(test_images[i].astype(np.float32)/255.) 
            i+=1
            
        pred=model1.predict(np.array(images))
        pred=np.squeeze(pred)
        predictions = np.concatenate((predictions, pred))
        del images
        gc.collect()
        
predictions1 = predict1()

In [ ]:
print(predictions[:50])
print(predictions1[:50])

In [ ]:
from scipy.special import logit, expit
predictions = expit((0.5* logit(predictions) + (0.5 * logit(predictions1))))

In [ ]:
preds = 0.5*(predictions + predictions1)

In [ ]:
results = pd.read_csv('../input/siim-isic-melanoma-classification/sample_submission.csv')
results['target'] = preds
results.to_csv('effnet3.csv', header=True, index=False)